In [1]:
#import sys
#!{sys.executable} -m pip install --upgrade "scikit-learn>=1.6.1,<1.7" category_encoders shap lyft_fugue

In [107]:
#usual packages
import sklearn
import numpy as np
from IPython.display import IFrame
from sklearn.model_selection import GroupKFold
from datetime import date
import pandas as pd
from pandas import DataFrame
import subprocess
import sys

#lyft packages
import lyft_data_toolkit.db.write.df
from lyft_data_toolkit.db.read import Compute
from lyft_data_toolkit.db.read import Query
from lyft_data_toolkit.db.read.df import to_dataframe  
from lyft_data_toolkit.v1_to_v3 import hive
from lyft_data_toolkit.v1_to_v3 import presto
from lyft_data_toolkit.v1_to_v3 import spark
from lyft_data_toolkit.v1_to_v3 import trino

#other packages
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

### 1. Run/change parameters

In [108]:
launch_date = "2025-08-04"

In [47]:
# import os
# print(os.getcwd())

In [109]:
from datetime import datetime, timedelta

today = datetime.today().date()

# Get previous week's Monday
report_start_dt = today - timedelta(days=today.weekday() + 7)
# Get previous week's Sunday
report_end_dt = report_start_dt + timedelta(days=6)

# Output as strings
report_start_dt = report_start_dt.strftime('%Y-%m-%d')
report_end_dt = report_end_dt.strftime('%Y-%m-%d')

print(report_start_dt, report_end_dt)

2025-12-29 2026-01-04


In [110]:
## used for deletion when type the wrong value
# lift_lookup = lift_lookup[~((lift_lookup['week_num'] == 1) & (lift_lookup['year_num'] == 2026))]
# lift_lookup = lift_lookup.reset_index(drop=True)
# lift_lookup

In [111]:
lift_lookup = pd.read_csv("weekly_lift_data.csv")
### insert in new on demand data 
lift_lookup = pd.concat([
    lift_lookup,
    pd.DataFrame([{
        "week_num": 1, #######
        "year_num": 2026, ###
        "starting_week_date": "2025-12-29", #######
        "pre_existing_rider_lift": 0.00, #########
        "new_rider_pp_lift": 0.0281 #######
    }])
], ignore_index=True)

lift_lookup = lift_lookup.sort_values("starting_week_date").drop_duplicates(ignore_index=True).reset_index(drop=True)
lift_lookup
lift_lookup.to_csv("weekly_lift_data.csv",index =False)

In [112]:
lift_lookup

,week_num,year_num,starting_week_date,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0.0227,1.8330
1,33,2025,2025-08-11,0.0227,1.8330
2,34,2025,2025-08-18,0.0227,1.8330
3,35,2025,2025-08-25,0.0227,1.8330
4,36,2025,2025-09-01,0.0227,1.8330
5,37,2025,2025-09-08,0.0227,1.8330
6,38,2025,2025-09-15,0.0227,1.8330
7,39,2025,2025-09-22,0.0227,1.8330
8,40,2025,2025-09-29,0.0227,1.8330
9,41,2025,2025-10-06,0.0130,1.2870


In [7]:
# from datetime import datetime, timedelta

# today = datetime.today().date()

# # Get previous week's Monday
# report_start_dt = today - timedelta(days=today.weekday() + 14)
# # Get previous week's Sunday
# report_end_dt = report_start_dt + timedelta(days=6)

# # Output as strings
# report_start_dt = report_start_dt.strftime('%Y-%m-%d')
# report_end_dt = report_end_dt.strftime('%Y-%m-%d')

# print(report_start_dt, report_end_dt)

2025-12-22 2025-12-28


In [113]:
print(report_start_dt, report_end_dt)

2025-12-29 2026-01-04


### 2. Run Prediction

In [114]:

SQL ="""
--higherlimit
WITH pre_final_table AS (
SELECT DISTINCT 
       sub1.rider_lyft_id,
       sub1.business_program_user_id,
       sub1.business_program_type,
       sub1.business_program_type_cleaned,
       sub1.lyft_signed_up_at,
       sub1.bp_created_at,
       sub1.bp_activated_at,
       sub1.user_type,
       sub1.sub_user_type,
       sub2.first_ride_finished_at
  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1
  LEFT JOIN coco.dim_user sub2 
    ON sub1.rider_lyft_id = sub2.user_lyft_id
 WHERE sub1.is_business_ride = TRUE 
) 

,pre_final_table_v2 AS (
SELECT rider_lyft_id,
       business_program_user_id,
       business_program_type,
       business_program_type_cleaned,
       lyft_signed_up_at,
       bp_created_at,
       bp_activated_at,
       user_type,
       sub_user_type,
       first_ride_finished_at,
       ROW_NUMBER() OVER(PARTITION BY rider_lyft_id ORDER BY bp_activated_at DESC) rnk 
  FROM pre_final_table 
 WHERE TRUE
)

SELECT DATE_TRUNC('week',CAST(bp_activated_at AS DATE)) week,
       business_program_type_cleaned,
       CASE WHEN DATE_DIFF('day',first_ride_finished_at,bp_activated_at) > 0 THEN 'PP' ELSE 'NP' END AS activation_type, -- proxy for NP/PP
       COUNT(DISTINCT rider_lyft_id) count_activations
  FROM pre_final_table_v2
 WHERE TRUE 
   AND rnk = 1
 GROUP BY 1,2,3
"""  
df = pd.DataFrame()
df = trino.query(SQL, scheduled=True)

{"ts": "2026-01-08T18:27:52.273Z", "uuid": "0f5a4f68-9a11-4f66-a924-a6734911c6df", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "bb21bd2ea5", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 590591586, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590591586', 'query': "--higherlimit\nWITH pre_final_table AS (\nSELECT DISTINCT \n       sub1.rider_lyft_id,\n       sub1.business_program_user_id,\n       sub1.business_program_type,\n       sub1.business_program_type_cleaned,\n       sub1.lyft_signed_up_at,\n       sub1.bp_created_at,\n       sub1.bp_activated_at,\n       sub1.user_type,\n       sub1.sub_user_type,\n       sub2.first_ride_finished_at\n  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1\n  LEFT JOIN coco.dim_user sub2 \n    ON sub1.rider_lyft_id = sub2.user_lyft_id\n WHERE sub1.is_business_ride = TRUE \n) \n\n,pre_final_table_v2 AS (\nSELECT rider_lyft_id,\n       business_program_user_id,\n       business_program_type,\n       business_program_type_cleaned,\n       lyft_signed_up_at,\n       bp_created_at,\n       bp_activated_at,\n       user_type,\n       sub_us

In [115]:
df.columns

Index(['week', 'business_program_type_cleaned', 'activation_type',
       'count_activations'],
      dtype='object')

In [116]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()

print(df_pivot)

business_program_type_cleaned       week activation_type  managed  organic
0                             2021-08-09              NP       13        1
1                             2021-08-09              PP      946      871
2                             2021-08-16              NP       40        4
3                             2021-08-16              PP     2016     1764
4                             2021-08-23              NP       17        3
..                                   ...             ...      ...      ...
457                           2025-12-22              PP      180      689
458                           2025-12-29              NP       34       19
459                           2025-12-29              PP      214      981
460                           2026-01-05              NP       38       25
461                           2026-01-05              PP      235      987

[462 rows x 4 columns]


In [117]:
np_df = df_pivot[df_pivot.activation_type == "NP"]
pp_df = df_pivot[df_pivot.activation_type == "PP"]

In [118]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from joblib import load

# launch_date = "2025-08-04"

np_df = df_pivot[df_pivot["activation_type"] == "NP"].copy()
pp_df = df_pivot[df_pivot["activation_type"] == "PP"].copy()

def add_lags(df):
    df = df.sort_values("week").reset_index(drop=True)
    df["organic_lag1"] = df["organic"].shift(1).fillna(method='bfill')
    df["organic_lag2"] = df["organic"].shift(2).fillna(method='bfill')
    return df

np_df = add_lags(np_df)
pp_df = add_lags(pp_df)

def predict_post_rewards(df, model_file, use_lag2=True):
    post_df = df[df["week"] >= launch_date].copy()
    if post_df.empty:
        print(f"No post-rewards data for {model_file}")
        return None
    
    #features = ["organic", "organic_lag1"]
    #if use_lag2:
        #features.append("organic_lag2")
    
    X_post = post_df["organic"]
    
    model = load(model_file)
    
    post_df["predicted_managed"] = model.predict(
        start=len(df[df["week"] < launch_date]),
        end=len(df)-1,
        exog=X_post
    )
    
    return post_df[["week", "managed", "predicted_managed"]]

np_post_preds = predict_post_rewards(np_df, "arimax_model_np.pkl", use_lag2=False)
pp_post_preds = predict_post_rewards(pp_df, "arimax_model_pp.pkl", use_lag2=True)

########## newly added nov 17, 25
np_post_preds['predicted_managed'] = np_post_preds['predicted_managed'].round(0)
pp_post_preds['predicted_managed'] = pp_post_preds['predicted_managed'].round(0)

if np_post_preds is not None:
    np_post_preds.to_csv("np_post_rewards_predictions.csv", index=False)
    print("NP post-rewards predictions exported to np_post_rewards_predictions.csv")

if pp_post_preds is not None:
    pp_post_preds.to_csv("pp_post_rewards_predictions.csv", index=False)
    print("PP post-rewards predictions exported to pp_post_rewards_predictions.csv")

print("NP Post-Rewards Predictions:")
print(np_post_preds)

print("PP Post-Rewards Predictions:")
print(pp_post_preds)


NP post-rewards predictions exported to np_post_rewards_predictions.csv
PP post-rewards predictions exported to pp_post_rewards_predictions.csv
NP Post-Rewards Predictions:
business_program_type_cleaned       week  managed  predicted_managed
208                           2025-08-04      134              133.0
209                           2025-08-11      119              143.0
210                           2025-08-18      130              142.0
211                           2025-08-25       92              122.0
212                           2025-09-01      108              110.0
213                           2025-09-08      151              161.0
214                           2025-09-15      171              161.0
215                           2025-09-22      150              144.0
216                           2025-09-29      169              126.0
217                           2025-10-06      196              152.0
218                           2025-10-13      146              168.0

### 3. calculate additional acitivation percentage - get incremental part into dataframe

In [119]:
np_preds = np_post_preds.reset_index(drop= True)
pp_preds = pp_post_preds.reset_index(drop = True)

In [120]:
np_preds['addl_activations'] = ((np_preds['managed'] - np_preds['predicted_managed']).clip(lower=0)).astype(int)
pp_preds['addl_activations'] = ((pp_preds['managed'] - pp_preds['predicted_managed']).clip(lower=0)).astype(int)

In [121]:
np_preds['perc_addl_activations'] = np_preds['addl_activations']/np_preds['managed']
pp_preds['perc_addl_activations'] = pp_preds['addl_activations']/pp_preds['managed']

In [122]:
np_preds


business_program_type_cleaned,week,managed,predicted_managed,addl_activations,perc_addl_activations
0,2025-08-04,134,133.0,1,0.007463
1,2025-08-11,119,143.0,0,0.000000
2,2025-08-18,130,142.0,0,0.000000
3,2025-08-25,92,122.0,0,0.000000
4,2025-09-01,108,110.0,0,0.000000
5,2025-09-08,151,161.0,0,0.000000
6,2025-09-15,171,161.0,10,0.058480
7,2025-09-22,150,144.0,6,0.040000
8,2025-09-29,169,126.0,43,0.254438
9,2025-10-06,196,152.0,44,0.224490


### 5. to build pre existing, new user pp, np rides table

#### 5.1 get pre existing data

In [138]:
pre_existing_sql  = f"""
-- higherlimit
SELECT week(requested_at) as week_num, year_of_week(requested_at) as year_num, 
DATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, 
date_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,
'pre_existing' as user_type,
count(distinct ride_id) as num_rides_pre_existing,
sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing
FROM shuninga.causal_analysis_bp_raw_data_step_4 
 WHERE user_type = 'pre-existing-user'
   AND most_used_business_program_type_cleaned = 'managed' 
  and date(requested_at) between date'{launch_date}' and date'{report_end_dt}'
  group by 1,2,3,4,5
  order by 1,2
  """ 
pre_exisiting_rides = pd.DataFrame()
pre_exisiting_rides = trino.query(pre_existing_sql, scheduled= True)

{"ts": "2026-01-08T18:56:35.888Z", "uuid": "6c7c5996-4fe4-405f-b6dc-e32a6900e1eb", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "bb21bd2ea5", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 590603606, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590603606', 'query': "-- higherlimit\nSELECT week(requested_at) as week_num, year_of_week(requested_at) as year_num, \nDATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, \ndate_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,\n'pre_existing' as user_type,\ncount(distinct ride_id) as num_rides_pre_existing,\nsum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing\nFROM shuninga.causal_analysis_bp_raw_data_step_4 \n WHERE user_type = 'pre-existing-user'\n   AND most_used_business_program_type_cleaned = 'managed' \n  and date(requested_at) between date'2025-08-04' and date'2026-01-04'\n  group by 1,2,3,4,5\n  order by 1,2"}
s3://mozartgateway-production-iad/mz/cmd/results/590603606/


In [139]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   week_num                22 non-null     int64         
 1   year_num                22 non-null     int64         
 2   starting_week_date      22 non-null     datetime64[ns]
 3   post_reward_week_num    22 non-null     int64         
 4   user_type               22 non-null     object        
 5   num_rides_pre_existing  22 non-null     int64         
 6   cum_rides_pre_existing  22 non-null     int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.3+ KB


In [140]:
pre_exisiting_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing
0,1,2026,2025-12-29,21,pre_existing,42009,1580830
1,32,2025,2025-08-04,0,pre_existing,72279,72279
2,33,2025,2025-08-11,1,pre_existing,71144,143423
3,34,2025,2025-08-18,2,pre_existing,70997,214420
4,35,2025,2025-08-25,3,pre_existing,68724,283144
5,36,2025,2025-09-01,4,pre_existing,64865,348009
6,37,2025,2025-09-08,5,pre_existing,79603,427612
7,38,2025,2025-09-15,6,pre_existing,82999,510611
8,39,2025,2025-09-22,7,pre_existing,78045,588656
9,40,2025,2025-09-29,8,pre_existing,75585,664241


In [141]:
lift_lookup["starting_week_date"] = pd.to_datetime(lift_lookup["starting_week_date"])
pre_exisiting_rides = pd.merge(
    pre_exisiting_rides,
    lift_lookup[["starting_week_date", "pre_existing_rider_lift"]],
    on="starting_week_date",
    how="left" 
)
pre_exisiting_rides.drop_duplicates(ignore_index=True)

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift
0,1,2026,2025-12-29,21,pre_existing,42009,1580830,0.0000
1,32,2025,2025-08-04,0,pre_existing,72279,72279,0.0227
2,33,2025,2025-08-11,1,pre_existing,71144,143423,0.0227
3,34,2025,2025-08-18,2,pre_existing,70997,214420,0.0227
4,35,2025,2025-08-25,3,pre_existing,68724,283144,0.0227
5,36,2025,2025-09-01,4,pre_existing,64865,348009,0.0227
6,37,2025,2025-09-08,5,pre_existing,79603,427612,0.0227
7,38,2025,2025-09-15,6,pre_existing,82999,510611,0.0227
8,39,2025,2025-09-22,7,pre_existing,78045,588656,0.0227
9,40,2025,2025-09-29,8,pre_existing,75585,664241,0.0227


In [142]:
pre_exisiting_rides['pre_existing_incr_rides'] =  (1-1/(1+pre_exisiting_rides['pre_existing_rider_lift']))*pre_exisiting_rides['num_rides_pre_existing']
pre_exisiting_rides['pre_existing_incr_rides'] = pre_exisiting_rides['pre_existing_incr_rides'].round(0) #######
pre_exisiting_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides
0,1,2026,2025-12-29,21,pre_existing,42009,1580830,0.0000,0.0
1,32,2025,2025-08-04,0,pre_existing,72279,72279,0.0227,1604.0
2,33,2025,2025-08-11,1,pre_existing,71144,143423,0.0227,1579.0
3,34,2025,2025-08-18,2,pre_existing,70997,214420,0.0227,1576.0
4,35,2025,2025-08-25,3,pre_existing,68724,283144,0.0227,1525.0
5,36,2025,2025-09-01,4,pre_existing,64865,348009,0.0227,1440.0
6,37,2025,2025-09-08,5,pre_existing,79603,427612,0.0227,1767.0
7,38,2025,2025-09-15,6,pre_existing,82999,510611,0.0227,1842.0
8,39,2025,2025-09-22,7,pre_existing,78045,588656,0.0227,1732.0
9,40,2025,2025-09-29,8,pre_existing,75585,664241,0.0227,1678.0


#### 5.2 get np and pp data

In [105]:
########## pp new user 
# np_preds 
# pp_preds
np_preds

business_program_type_cleaned,week,managed,predicted_managed,addl_activations,perc_addl_activations
0,2025-08-04,134,133.0,1,0.007463
1,2025-08-11,119,143.0,0,0.000000
2,2025-08-18,130,142.0,0,0.000000
3,2025-08-25,92,122.0,0,0.000000
4,2025-09-01,108,110.0,0,0.000000
5,2025-09-08,151,161.0,0,0.000000
6,2025-09-15,171,161.0,10,0.058480
7,2025-09-22,150,144.0,6,0.040000
8,2025-09-29,169,126.0,43,0.254438
9,2025-10-06,196,152.0,44,0.224490


In [143]:
new_user_pp_sql = f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year_of_week(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user pp' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_pp,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-PP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_pp_rides = pd.DataFrame()
new_pp_rides = trino.query(new_user_pp_sql,scheduled =True)
new_pp_rides

{"ts": "2026-01-08T19:04:06.252Z", "uuid": "e2223f3a-b1cd-4a8f-a4d5-eb5871d403e4", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "bb21bd2ea5", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}
/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:330: UserWarning: [lyft_stats.client] stats dropped due to err=ConnectionResetError(104, 'Connection reset by peer')
  warnings.warn(f"[{__name__}] stats dropped due to {err=}")


Created Mozart command
{'mozart_id': 590607131, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590607131', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2026-01-04', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year_of_week(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user pp' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_pp,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-PP'\n  AND d.most_used_business_program

/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:347: UserWarning: Attempting to establish a new TCP stats connection
  warnings.warn("Attempting to establish a new TCP stats connection")


s3://mozartgateway-production-iad/mz/cmd/results/590607131/


,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp
0,32,2025,2025-08-04,0,new user pp,2511,2511
1,33,2025,2025-08-11,1,new user pp,3115,5626
2,34,2025,2025-08-18,2,new user pp,3784,9410
3,35,2025,2025-08-25,3,new user pp,4127,13537
4,36,2025,2025-09-01,4,new user pp,4042,17579
5,37,2025,2025-09-08,5,new user pp,6303,23882
6,38,2025,2025-09-15,6,new user pp,7756,31638
7,39,2025,2025-09-22,7,new user pp,7539,39177
8,40,2025,2025-09-29,8,new user pp,7670,46847
9,41,2025,2025-10-06,9,new user pp,9497,56344


In [144]:
pp_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   week                   23 non-null     datetime64[ns]
 1   managed                23 non-null     int64         
 2   predicted_managed      23 non-null     float64       
 3   addl_activations       23 non-null     int64         
 4   perc_addl_activations  23 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 1.0 KB


In [145]:
new_pp_rides = pd.merge(new_pp_rides,
                        pp_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 

new_pp_rides = pd.merge(new_pp_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift','new_rider_pp_lift']],
                        on = 'starting_week_date',
                        how = 'left')
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0,new user pp,2511,2511,2025-08-04,0.000000,0.0227,1.8330
1,33,2025,2025-08-11,1,new user pp,3115,5626,2025-08-11,0.000000,0.0227,1.8330
2,34,2025,2025-08-18,2,new user pp,3784,9410,2025-08-18,0.051710,0.0227,1.8330
3,35,2025,2025-08-25,3,new user pp,4127,13537,2025-08-25,0.000000,0.0227,1.8330
4,36,2025,2025-09-01,4,new user pp,4042,17579,2025-09-01,0.000000,0.0227,1.8330
5,37,2025,2025-09-08,5,new user pp,6303,23882,2025-09-08,0.084162,0.0227,1.8330
6,38,2025,2025-09-15,6,new user pp,7756,31638,2025-09-15,0.111246,0.0227,1.8330
7,39,2025,2025-09-22,7,new user pp,7539,39177,2025-09-22,0.076302,0.0227,1.8330
8,40,2025,2025-09-29,8,new user pp,7670,46847,2025-09-29,0.086082,0.0227,1.8330
9,41,2025,2025-10-06,9,new user pp,9497,56344,2025-10-06,0.162745,0.0130,1.2870


In [146]:
### calculate incremental rides
new_pp_rides['pp_counterfactual_rides'] = ((new_pp_rides['num_rides_new_pp']*(1-new_pp_rides['perc_addl_activations']))).round(0)
new_pp_rides['pp_addl_rides'] = new_pp_rides['num_rides_new_pp']- new_pp_rides['pp_counterfactual_rides'] 
new_pp_rides['pp_incre_rides_from_counterfactual'] = ((1-1/(1+new_pp_rides['pre_existing_rider_lift']))*new_pp_rides['pp_counterfactual_rides']).round(0)
new_pp_rides['pp_incre_rides_from_addl'] = ((1-1/(1+new_pp_rides['new_rider_pp_lift']))*new_pp_rides['pp_addl_rides']).round(0)

In [147]:
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift,pp_counterfactual_rides,pp_addl_rides,pp_incre_rides_from_counterfactual,pp_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user pp,2511,2511,2025-08-04,0.000000,0.0227,1.8330,2511.0,0.0,56.0,0.0
1,33,2025,2025-08-11,1,new user pp,3115,5626,2025-08-11,0.000000,0.0227,1.8330,3115.0,0.0,69.0,0.0
2,34,2025,2025-08-18,2,new user pp,3784,9410,2025-08-18,0.051710,0.0227,1.8330,3588.0,196.0,80.0,127.0
3,35,2025,2025-08-25,3,new user pp,4127,13537,2025-08-25,0.000000,0.0227,1.8330,4127.0,0.0,92.0,0.0
4,36,2025,2025-09-01,4,new user pp,4042,17579,2025-09-01,0.000000,0.0227,1.8330,4042.0,0.0,90.0,0.0
5,37,2025,2025-09-08,5,new user pp,6303,23882,2025-09-08,0.084162,0.0227,1.8330,5773.0,530.0,128.0,343.0
6,38,2025,2025-09-15,6,new user pp,7756,31638,2025-09-15,0.111246,0.0227,1.8330,6893.0,863.0,153.0,558.0
7,39,2025,2025-09-22,7,new user pp,7539,39177,2025-09-22,0.076302,0.0227,1.8330,6964.0,575.0,155.0,372.0
8,40,2025,2025-09-29,8,new user pp,7670,46847,2025-09-29,0.086082,0.0227,1.8330,7010.0,660.0,156.0,427.0
9,41,2025,2025-10-06,9,new user pp,9497,56344,2025-10-06,0.162745,0.0130,1.2870,7951.0,1546.0,102.0,870.0


#### get new user np data

In [154]:
new_user_np_sql=f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year_of_week(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user np' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_np,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-NP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_np_rides = pd.DataFrame()
new_np_rides = trino.query(new_user_np_sql,scheduled =True)
new_np_rides

{"ts": "2026-01-08T19:06:53.158Z", "uuid": "83f5a1b1-d3c7-4ee1-9c8e-6480301bc09d", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "bb21bd2ea5", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 590608710, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590608710', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2026-01-04', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year_of_week(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user np' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_np,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-NP'\n  AND d.most_used_business_program

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np
0,32,2025,2025-08-04,0,new user np,173,173
1,33,2025,2025-08-11,1,new user np,356,529
2,34,2025,2025-08-18,2,new user np,480,1009
3,35,2025,2025-08-25,3,new user np,478,1487
4,36,2025,2025-09-01,4,new user np,552,2039
5,37,2025,2025-09-08,5,new user np,740,2779
6,38,2025,2025-09-15,6,new user np,936,3715
7,39,2025,2025-09-22,7,new user np,945,4660
8,40,2025,2025-09-29,8,new user np,957,5617
9,41,2025,2025-10-06,9,new user np,1225,6842


In [155]:
new_np_rides = pd.merge(new_np_rides,
                        np_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 
new_np_rides = pd.merge(new_np_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift']],
                        on = 'starting_week_date',
                        how = 'left')

In [156]:
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.007463,0.0227
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0227
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0227
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0227
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0227
5,37,2025,2025-09-08,5,new user np,740,2779,2025-09-08,0.000000,0.0227
6,38,2025,2025-09-15,6,new user np,936,3715,2025-09-15,0.058480,0.0227
7,39,2025,2025-09-22,7,new user np,945,4660,2025-09-22,0.040000,0.0227
8,40,2025,2025-09-29,8,new user np,957,5617,2025-09-29,0.254438,0.0227
9,41,2025,2025-10-06,9,new user np,1225,6842,2025-10-06,0.224490,0.0130


In [157]:
### calculate incremental rides
new_np_rides['np_counterfactual_rides'] = (new_np_rides['num_rides_new_np']*(1-new_np_rides['perc_addl_activations'])).round(0)
new_np_rides['np_addl_rides'] = new_np_rides['num_rides_new_np']- new_np_rides['np_counterfactual_rides'] 
new_np_rides['np_incre_rides_from_counterfactual'] = ((1-1/(1+new_np_rides['pre_existing_rider_lift']))*new_np_rides['np_counterfactual_rides']).round(0)
new_np_rides['np_incre_rides_from_addl'] = new_np_rides['np_addl_rides']
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.007463,0.0227,172.0,1.0,4.0,1.0
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0227,356.0,0.0,8.0,0.0
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0227,480.0,0.0,11.0,0.0
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0227,478.0,0.0,11.0,0.0
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0227,552.0,0.0,12.0,0.0
5,37,2025,2025-09-08,5,new user np,740,2779,2025-09-08,0.000000,0.0227,740.0,0.0,16.0,0.0
6,38,2025,2025-09-15,6,new user np,936,3715,2025-09-15,0.058480,0.0227,881.0,55.0,20.0,55.0
7,39,2025,2025-09-22,7,new user np,945,4660,2025-09-22,0.040000,0.0227,907.0,38.0,20.0,38.0
8,40,2025,2025-09-29,8,new user np,957,5617,2025-09-29,0.254438,0.0227,714.0,243.0,16.0,243.0
9,41,2025,2025-10-06,9,new user np,1225,6842,2025-10-06,0.224490,0.0130,950.0,275.0,12.0,275.0


In [158]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 0 to 21
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   week_num                 22 non-null     int64         
 1   year_num                 22 non-null     int64         
 2   starting_week_date       22 non-null     datetime64[ns]
 3   post_reward_week_num     22 non-null     int64         
 4   user_type                22 non-null     object        
 5   num_rides_pre_existing   22 non-null     int64         
 6   cum_rides_pre_existing   22 non-null     int64         
 7   pre_existing_rider_lift  22 non-null     float64       
 8   pre_existing_incr_rides  22 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 1.7+ KB


In [159]:
# #### get final result with all incrementality

# incremental_all_rides = pd.merge(
#                         pre_exisiting_rides[['week_num','year_num','starting_week_date','post_reward_week_num','num_rides_pre_existing','cum_rides_pre_existing','pre_existing_rider_lift','pre_existing_incr_rides']],
#                         new_pp_rides[['starting_week_date','num_rides_new_pp','cum_rides_new_pp','new_rider_pp_lift','pp_counterfactual_rides','pp_addl_rides','pp_incre_rides_from_counterfactual','pp_incre_rides_from_addl']],
#                         on = 'starting_week_date',
#                         how = 'left'
# )

# incremental_all_rides = pd.merge(
#     incremental_all_rides,
#     new_np_rides[['starting_week_date','num_rides_new_np','cum_rides_new_np','np_counterfactual_rides','np_addl_rides','np_incre_rides_from_counterfactual','np_incre_rides_from_addl']],
#     on = 'starting_week_date',
#     how = 'left'
# )
# incremental_all_rides['total_incre_rides']= incremental_all_rides['pre_existing_incr_rides']+ incremental_all_rides['pp_incre_rides_from_counterfactual']+ incremental_all_rides['pp_incre_rides_from_addl']+incremental_all_rides['np_incre_rides_from_counterfactual']+incremental_all_rides['np_incre_rides_from_addl']
# incremental_all_rides['cum_total_incre_rides'] = incremental_all_rides['total_incre_rides'].cumsum()
# incremental_all_rides['total_observed_rides'] = incremental_all_rides['num_rides_pre_existing']+incremental_all_rides['num_rides_new_pp']+incremental_all_rides['num_rides_new_np']
# incremental_all_rides['cum_total_observed_rides'] = incremental_all_rides['total_observed_rides'].cumsum()
# incremental_all_rides['cum_baseline_rides'] = incremental_all_rides['cum_total_observed_rides']-incremental_all_rides['cum_total_incre_rides']
# incremental_all_rides['progress_t_target'] = incremental_all_rides['cum_total_incre_rides']/713894
# incremental_all_rides = incremental_all_rides.drop_duplicates(ignore_index = True)

In [166]:
#### get final result with all incrementality
incremental_all_rides = pd.merge(
                        pre_exisiting_rides[['week_num','year_num','starting_week_date','post_reward_week_num','num_rides_pre_existing','cum_rides_pre_existing','pre_existing_rider_lift','pre_existing_incr_rides']],
                        new_pp_rides[['starting_week_date','num_rides_new_pp','cum_rides_new_pp','new_rider_pp_lift','pp_counterfactual_rides','pp_addl_rides','pp_incre_rides_from_counterfactual','pp_incre_rides_from_addl']],
                        on = 'starting_week_date',
                        how = 'left'
)

incremental_all_rides = pd.merge(
    incremental_all_rides,
    new_np_rides[['starting_week_date','num_rides_new_np','cum_rides_new_np','np_counterfactual_rides','np_addl_rides','np_incre_rides_from_counterfactual','np_incre_rides_from_addl']],
    on = 'starting_week_date',
    how = 'left'
)
incremental_all_rides['total_incre_rides']= incremental_all_rides['pre_existing_incr_rides']+ incremental_all_rides['pp_incre_rides_from_counterfactual']+ incremental_all_rides['pp_incre_rides_from_addl']+incremental_all_rides['np_incre_rides_from_counterfactual']+incremental_all_rides['np_incre_rides_from_addl']

incremental_all_rides['total_observed_rides'] = incremental_all_rides['num_rides_pre_existing']+incremental_all_rides['num_rides_new_pp']+incremental_all_rides['num_rides_new_np']

incremental_all_rides = incremental_all_rides.drop_duplicates(ignore_index = True)
incremental_all_rides = incremental_all_rides.sort_values('starting_week_date').reset_index(drop=True)
incremental_all_rides['cum_total_incre_rides'] = incremental_all_rides['total_incre_rides'].cumsum()
incremental_all_rides['cum_total_observed_rides'] = incremental_all_rides['total_observed_rides'].cumsum()
incremental_all_rides['cum_baseline_rides'] = incremental_all_rides['cum_total_observed_rides']-incremental_all_rides['cum_total_incre_rides']
incremental_all_rides['progress_t_target'] = incremental_all_rides['cum_total_incre_rides']/713894

In [167]:
incremental_all_rides

,week_num,year_num,starting_week_date,post_reward_week_num,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides,num_rides_new_pp,cum_rides_new_pp,...,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl,total_incre_rides,total_observed_rides,cum_total_incre_rides,cum_total_observed_rides,cum_baseline_rides,progress_t_target
0,32,2025,2025-08-04,0,72279,72279,0.0227,1604.0,2511,2511,...,172.0,1.0,4.0,1.0,1665.0,74963,1665.0,74963,73298.0,0.002332
1,33,2025,2025-08-11,1,71144,143423,0.0227,1579.0,3115,5626,...,356.0,0.0,8.0,0.0,1656.0,74615,3321.0,149578,146257.0,0.004652
2,34,2025,2025-08-18,2,70997,214420,0.0227,1576.0,3784,9410,...,480.0,0.0,11.0,0.0,1794.0,75261,5115.0,224839,219724.0,0.007165
3,35,2025,2025-08-25,3,68724,283144,0.0227,1525.0,4127,13537,...,478.0,0.0,11.0,0.0,1628.0,73329,6743.0,298168,291425.0,0.009445
4,36,2025,2025-09-01,4,64865,348009,0.0227,1440.0,4042,17579,...,552.0,0.0,12.0,0.0,1542.0,69459,8285.0,367627,359342.0,0.011605
5,37,2025,2025-09-08,5,79603,427612,0.0227,1767.0,6303,23882,...,740.0,0.0,16.0,0.0,2254.0,86646,10539.0,454273,443734.0,0.014763
6,38,2025,2025-09-15,6,82999,510611,0.0227,1842.0,7756,31638,...,881.0,55.0,20.0,55.0,2628.0,91691,13167.0,545964,532797.0,0.018444
7,39,2025,2025-09-22,7,78045,588656,0.0227,1732.0,7539,39177,...,907.0,38.0,20.0,38.0,2317.0,86529,15484.0,632493,617009.0,0.021689
8,40,2025,2025-09-29,8,75585,664241,0.0227,1678.0,7670,46847,...,714.0,243.0,16.0,243.0,2520.0,84212,18004.0,716705,698701.0,0.025219
9,41,2025,2025-10-06,9,80892,745133,0.0130,1038.0,9497,56344,...,950.0,275.0,12.0,275.0,2297.0,91614,20301.0,808319,788018.0,0.028437


In [168]:
incremental_all_rides.to_csv('incremtal_all_rides.csv',index= False)

In [169]:
incremental_all_rides.to_hive_table(table_name='bp_incremental_all_rides', schema_name='shuninga', if_exists = 'replace')

{"ts": "2026-01-08T19:45:54.792Z", "uuid": "11f3a606-ab96-41e1-879c-b1072c5b1b8d", "name": "lyft_data_toolkit.v1_to_v3.hive", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/hive.py:32", "revision": "bb21bd2ea5", "msg": "DEPRECATION WARNING: to_hive_table method is outdated.Please use the new write_to_table function in the lyft_data_toolkit package."}
{"ts": "2026-01-08T19:45:55.322Z", "uuid": "bd527299-61fb-4e31-8e16-3d4980b1ea21", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "bb21bd2ea5", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 590625405, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590625405', 'query': "SHOW TABLES FROM shuninga LIKE 'bp_incremental_all_rides'"}


/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:330: UserWarning: [lyft_stats.client] stats dropped due to err=ConnectionResetError(104, 'Connection reset by peer')
  warnings.warn(f"[{__name__}] stats dropped due to {err=}")
/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:347: UserWarning: Attempting to establish a new TCP stats connection
  warnings.warn("Attempting to establish a new TCP stats connection")
{"ts": "2026-01-08T19:46:26.893Z", "uuid": "bc3633b8-0ea0-4125-aeb2-cd02c92361fc", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "bb21bd2ea5", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 590625637, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590625637', 'query': 'DROP TABLE IF EXISTS shuninga.bp_incremental_all_rides'}


{"ts": "2026-01-08T19:48:58.158Z", "uuid": "78fe0151-6bb8-48b4-b000-2c1c80c338ac", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 101, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "bb21bd2ea5", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 590626471, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/590626471', 'query': "CREATE EXTERNAL TABLE IF NOT EXISTS shuninga.bp_incremental_all_rides (week_num bigint, year_num bigint, starting_week_date timestamp, post_reward_week_num bigint, num_rides_pre_existing bigint, cum_rides_pre_existing bigint, pre_existing_rider_lift double, pre_existing_incr_rides double, num_rides_new_pp bigint, cum_rides_new_pp bigint, new_rider_pp_lift double, pp_counterfactual_rides double, pp_addl_rides double, pp_incre_rides_from_counterfactual double, pp_incre_rides_from_addl double, num_rides_new_np bigint, cum_rides_new_np bigint, np_counterfactual_rides double, np_addl_rides double, np_incre_rides_from_counterfactual double, np_incre_rides_from_addl double, total_incre_rides double, total_observed_rides bigint, cum_total_incre_rides double, cum_total_observed_rides bigint, cum_baseline_rides double,

In [170]:
incremental_all_rides

,week_num,year_num,starting_week_date,post_reward_week_num,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides,num_rides_new_pp,cum_rides_new_pp,...,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl,total_incre_rides,total_observed_rides,cum_total_incre_rides,cum_total_observed_rides,cum_baseline_rides,progress_t_target
0,32,2025,2025-08-04,0,72279,72279,0.0227,1604.0,2511,2511,...,172.0,1.0,4.0,1.0,1665.0,74963,1665.0,74963,73298.0,0.002332
1,33,2025,2025-08-11,1,71144,143423,0.0227,1579.0,3115,5626,...,356.0,0.0,8.0,0.0,1656.0,74615,3321.0,149578,146257.0,0.004652
2,34,2025,2025-08-18,2,70997,214420,0.0227,1576.0,3784,9410,...,480.0,0.0,11.0,0.0,1794.0,75261,5115.0,224839,219724.0,0.007165
3,35,2025,2025-08-25,3,68724,283144,0.0227,1525.0,4127,13537,...,478.0,0.0,11.0,0.0,1628.0,73329,6743.0,298168,291425.0,0.009445
4,36,2025,2025-09-01,4,64865,348009,0.0227,1440.0,4042,17579,...,552.0,0.0,12.0,0.0,1542.0,69459,8285.0,367627,359342.0,0.011605
5,37,2025,2025-09-08,5,79603,427612,0.0227,1767.0,6303,23882,...,740.0,0.0,16.0,0.0,2254.0,86646,10539.0,454273,443734.0,0.014763
6,38,2025,2025-09-15,6,82999,510611,0.0227,1842.0,7756,31638,...,881.0,55.0,20.0,55.0,2628.0,91691,13167.0,545964,532797.0,0.018444
7,39,2025,2025-09-22,7,78045,588656,0.0227,1732.0,7539,39177,...,907.0,38.0,20.0,38.0,2317.0,86529,15484.0,632493,617009.0,0.021689
8,40,2025,2025-09-29,8,75585,664241,0.0227,1678.0,7670,46847,...,714.0,243.0,16.0,243.0,2520.0,84212,18004.0,716705,698701.0,0.025219
9,41,2025,2025-10-06,9,80892,745133,0.0130,1038.0,9497,56344,...,950.0,275.0,12.0,275.0,2297.0,91614,20301.0,808319,788018.0,0.028437


### extra cells 

In [ ]:
# del lift_lookup

In [ ]:
# columns = [
#     "week_num", # begining week number of biweekly              
#     "year_num",  # beginning year number of the biweekly period             
#     "starting_week_date",    
#     "pre_existing_rider_lift", # get it from the tool
#     "new_rider_pp_lift"
# ]

# lift_lookup = pd.DataFrame(columns=columns)

# #  Function to append new biweekly data
# def add_biweekly_entry(df, week_num, year_num, starting_week_date, pre_existing_rider_lift, new_rider_pp_lift):
#     new_row = {
#         "week_num": int(week_num),
#         "year_num": int(year_num),
#         "starting_week_date": pd.to_datetime(starting_week_date),
#         "pre_existing_rider_lift": float(pre_existing_rider_lift),
#         "new_rider_pp_lift": float(new_rider_pp_lift)
#     }
#     return df.append(new_row, ignore_index=True)

# # do this biweekly to update the dataframe
# lift_lookup = add_biweekly_entry(lift_lookup, week_num=44, year_num=2025, starting_week_date="2025-11-02", pre_existing_rider_lift= 0.6, new_rider_pp_lift=1)

# # optional
# lift_lookup.to_csv("biweekly_lift_data.csv", index=False)

# lift_lookup